# eidein

Interactive tool for dimensionality reduction of astronomical data and their visualisation.

## TODO List

1. implement BNN and prediction uncertanties
1. subset (lasso) selection (widgets.SelectMultiple)
1. divide features (for dimensionality reduction) and spectra (for visualisation, ploting function)
1. scale to Big Data

In [1]:
%matplotlib widget

from astropy.modeling import models, fitting
import h5py
import matplotlib.pyplot as plt
import numpy as np

from eidein import Eidein
import plot

## Data

In [2]:
SIZE = 2048

In [3]:
# TODO should I take only those with high S/N?
with h5py.File("data/dr12q_superset.hdf5", "r") as datafile:
    n = datafile["id"].shape[0]
    rnd_idx = np.random.randint(n, size=SIZE)
    ids = datafile["id"][:][rnd_idx]
    X = datafile["X"][:][rnd_idx]
    z = datafile["z_vi"][:][rnd_idx]
ids.shape, X.shape, z.shape

((2048, 3), (2048, 3752), (2048,))

In [4]:
rnd_i = np.random.randint(SIZE)
fig, ax = plt.subplots()
plot.spectrum(ax, X[rnd_i], label="Z_VI = " + str(z[rnd_i]))
plot.redshift(ax, z[rnd_i])
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
Eidein(ids, X, z)

Eidein(children=(Tab(children=(interactive(children=(Checkbox(value=False, description='whiten'), Dropdown(des…